In [4]:
import os.path

#from IPython.display import display
import  pandas as pd
from collections import Counter
#from src.dataset import MelanomaDataset,data_preprocessing,Normalize
from sklearn.model_selection import train_test_split
import shutil
from torchvision import models, transforms
import os

In [3]:

def copyfile_to_dir(fpath, dirPath):
    shutil.copyfile(fpath, dirPath + '/' + fpath.split('/')[-1])

In [5]:
print("nn..gg..mm")

nn..gg..mm


In [4]:
raw_data_csv_path="../dataset/HAM10000_metadata.csv"
ham_img_dataset_path="../dataset/preprocessed/ham1000"

In [5]:
raw_data=pd.read_csv(raw_data_csv_path)


In [6]:
raw_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [7]:
raw_data.columns

Index(['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization',
       'dataset'],
      dtype='object')

In [8]:
non_numeric_cols=['sex', 'localization','dx']

In [9]:
for col in non_numeric_cols:
    display(pd.DataFrame(Counter(raw_data[col]).items()))

,0,1
0,male,5406
1,female,4552
2,unknown,57


,0,1
0,scalp,128
1,ear,56
2,face,745
3,back,2192
4,trunk,1404
5,chest,407
6,upper extremity,1118
7,abdomen,1022
8,unknown,234
9,lower extremity,2077


,0,1
0,bkl,1099
1,nv,6705
2,df,115
3,mel,1113
4,vasc,142
5,bcc,514
6,akiec,327


In [3]:
skin_path="../dataset/raw/skin lesion/metadata.csv"

In [4]:
skin=pd.read_csv(skin_path)

In [5]:
skin.head()

,image_id,dx,age,sex,localization
0,ISIC_0000002,mel,60.0,female,upper extremity
1,ISIC_0000004,mel,80.0,male,posterior torso
2,ISIC_0000013,mel,30.0,female,posterior torso
3,ISIC_0000022,mel,55.0,female,lower extremity
4,ISIC_0000026,mel,30.0,female,lower extremity


In [6]:
Counter(skin['localization'])

Counter({'upper extremity': 337,
         'posterior torso': 33,
         'lower extremity': 478,
         'anterior torso': 845,
         nan: 207,
         'lateral torso': 4,
         'head/neck': 1111,
         'palms/soles': 67,
         'oral/genital': 19})

In [8]:
skin=skin.dropna()

In [7]:
for i in range(len(skin)):
    if skin.iloc[i][1] == "mel":
        skin.at[i, 'dx'] = 1
    else:
         skin.at[i, 'dx'] = 0

In [16]:
skin.head()

,image_id,dx,age,sex,localization
0,ISIC_0000002,1,60.0,female,upper extremity
1,ISIC_0000004,1,80.0,male,posterior torso
2,ISIC_0000013,1,30.0,female,posterior torso
3,ISIC_0000022,1,55.0,female,lower extremity
4,ISIC_0000026,1,30.0,female,lower extremity


In [9]:
Counter(skin['dx'])

Counter({1: 813, 0: 2049})

In [10]:
skin.to_csv("../dataset/raw/newfile2.csv")

In [28]:
dummy_cols = ['localization', 'sex']

for col in dummy_cols:
    dummy = pd.get_dummies(skin[col], prefix=col, drop_first=False)
    skin = pd.concat([skin, dummy], axis=1)

skin.drop(dummy_cols, axis=1, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'col'

In [5]:
raw_data_csv_path="../dataset/HAM10000_metadata.csv"
ham_data=pd.read_csv(raw_data_csv_path)

In [6]:
ham_data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [9]:
del ham_data['lesion_id']
del ham_data['dx_type']
del ham_data['dataset']

In [10]:
for i in range(ham_data.shape[0]):
        if ham_data.iloc[i][1] == "mel":
            ham_data.at[i, 'dx'] = 1
        else:
            ham_data.at[i, 'dx'] = 0

In [13]:
idx_to_remove = set()
for i in range(len(ham_data)):

    if ham_data.iloc[i][3] == "unknown":
     idx_to_remove.add(i)

for i in range(len(ham_data)):
    if ham_data.iloc[i][4] == "unknown":
        idx_to_remove.add(i)
ham_data = ham_data.drop(index=list(idx_to_remove))

In [14]:
ham_data.head()

,image_id,dx,age,sex,localization
0,ISIC_0027419,0,80.0,male,scalp
1,ISIC_0025030,0,80.0,male,scalp
2,ISIC_0026769,0,80.0,male,scalp
3,ISIC_0025661,0,80.0,male,scalp
4,ISIC_0031633,0,75.0,male,ear


In [15]:
ham_data.to_csv("../dataset/raw/newfile3.csv")

In [7]:
isic_img_path="../dataset/raw/ISIC 2020/train"

In [8]:
isic_path="../dataset/raw/ISIC 2020/ISIC_2020_Training_GroundTruth.csv"

In [9]:
isic=pd.read_csv(isic_path)

In [10]:
isic.head()


,image_name,target,sex,age_approx,anatom_site_general_challenge
0,ISIC_2637011,0,male,45.0,head/neck
1,ISIC_0015719,0,female,45.0,upper extremity
2,ISIC_0052212,0,female,50.0,lower extremity
3,ISIC_0068279,0,female,45.0,head/neck
4,ISIC_0074268,0,female,55.0,upper extremity


In [11]:
isic=isic.dropna()

In [12]:
isic.head()

,image_name,target,sex,age_approx,anatom_site_general_challenge
0,ISIC_2637011,0,male,45.0,head/neck
1,ISIC_0015719,0,female,45.0,upper extremity
2,ISIC_0052212,0,female,50.0,lower extremity
3,ISIC_0068279,0,female,45.0,head/neck
4,ISIC_0074268,0,female,55.0,upper extremity


In [13]:
c= list(isic.columns.values)
print(c)

['image_name', 'target', 'sex', 'age_approx', 'anatom_site_general_challenge']


In [14]:
isic.rename(columns={'image_name':'image_id','target':'dx','age_approx':'age' ,'anatom_site_general_challenge':'localization'},inplace=True)

In [15]:
isic.head()

,image_id,dx,sex,age,localization
0,ISIC_2637011,0,male,45.0,head/neck
1,ISIC_0015719,0,female,45.0,upper extremity
2,ISIC_0052212,0,female,50.0,lower extremity
3,ISIC_0068279,0,female,45.0,head/neck
4,ISIC_0074268,0,female,55.0,upper extremity


In [14]:
Counter(isic['sex'])

Counter({'male': 16788, 'female': 15743})

In [15]:
Counter(isic['age_approx'])

Counter({45.0: 4286,
         50.0: 4224,
         55.0: 3785,
         40.0: 3531,
         25.0: 1523,
         35.0: 2802,
         30.0: 2318,
         75.0: 974,
         70.0: 1949,
         65.0: 2500,
         60.0: 3201,
         80.0: 416,
         20.0: 642,
         85.0: 149,
         15.0: 132,
         90.0: 80,
         10.0: 17,
         0.0: 2})

In [16]:
Counter(isic['target'])

Counter({0: 31956, 1: 575})

In [17]:
Counter(isic['anatom_site_general_challenge'])

Counter({'head/neck': 1845,
         'upper extremity': 4963,
         'lower extremity': 8399,
         'torso': 16825,
         'palms/soles': 375,
         'oral/genital': 124})

In [ ]:
for i in range(len(isic)):
    if isic.iloc[0][4]==1:


#Last Dataset


In [5]:
raw_images_path="../dataset/raw/all_images"
all_images_csv_path="../dataset/raw/all_data.csv"

In [6]:
dataset=pd.read_csv(all_images_csv_path)

In [7]:
dataset=dataset.dropna()

In [8]:
loc=pd.get_dummies(dataset['location '],prefix="locatin", drop_first=False)
dataset = pd.concat([dataset, loc], axis=1)

In [9]:
gender=pd.get_dummies(dataset['gender'],prefix="gender", drop_first=False)
dataset = pd.concat([dataset, gender], axis=1)

In [10]:
dataset.shape
Counter(dataset['target'])

Counter({0: 42641, 1: 2497})

In [11]:
dataset

,image name,target,age,gender,location,locatin_abdomen,locatin_acral,locatin_anterior torso,locatin_back,locatin_chest,...,locatin_neck,locatin_oral/genital,locatin_palms/soles,locatin_posterior torso,locatin_scalp,locatin_torso,locatin_trunk,locatin_upper extremity,gender_female,gender_male
0,ISIC_2637011,0,45.0,male,head/neck,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,ISIC_0015719,0,45.0,female,upper extremity,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,ISIC_0052212,0,50.0,female,lower extremity,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,ISIC_0068279,0,45.0,female,head/neck,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,ISIC_0074268,0,55.0,female,upper extremity,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45754,ISIC_0062247,0,55.0,female,head/neck,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
45755,ISIC_0062257,0,75.0,male,head/neck,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
45756,ISIC_0062258,0,85.0,male,upper extremity,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
45757,ISIC_0062300,0,80.0,male,head/neck,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
del dataset['gender']



In [12]:
dataset.head()

,image name,target,age,location,locatin_abdomen,locatin_acral,locatin_anterior torso,locatin_back,locatin_chest,locatin_ear,...,locatin_neck,locatin_oral/genital,locatin_palms/soles,locatin_posterior torso,locatin_scalp,locatin_torso,locatin_trunk,locatin_upper extremity,gender_female,gender_male
0,ISIC_2637011,0,45.0,head/neck,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,ISIC_0015719,0,45.0,upper extremity,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,ISIC_0052212,0,50.0,lower extremity,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,ISIC_0068279,0,45.0,head/neck,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,ISIC_0074268,0,55.0,upper extremity,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [13]:
del dataset['location ']

In [14]:
train_all_data,test_all_data=train_test_split(dataset)

In [16]:
train_all_data.to_csv("../dataset/preprocessed/all_train.csv",index=False)
test_all_data.to_csv("../dataset/preprocessed/all_test.csv",index=False)

ashr7y 3mlty eh

In [19]:
preprocessed_data=ham_data_csv_preprocessing(raw_data_csv_path)

NameError: name 'ham_data_csv_preprocessing' is not defined

In [10]:
preprocessed_data.head()

#Counter(preprocessed_data['dx'])

,image_id,dx,age,localization_abdomen,localization_acral,localization_back,localization_chest,localization_ear,localization_face,localization_foot,localization_genital,localization_hand,localization_lower extremity,localization_neck,localization_scalp,localization_trunk,localization_upper extremity,sex_female,sex_male
0,ISIC_0027419,0,80.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
1,ISIC_0025030,0,80.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
2,ISIC_0026769,0,80.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,ISIC_0025661,0,80.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,ISIC_0031633,0,75.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [7]:
preprocessed_data.shape

(9761, 19)

In [8]:
Counter(preprocessed_data['dx'])

Counter({0: 8660, 1: 1101})

In [9]:
preprocessed_data.iloc[0][2:]

age                             80.0
localization_abdomen               0
localization_acral                 0
localization_back                  0
localization_chest                 0
localization_ear                   0
localization_face                  0
localization_foot                  0
localization_genital               0
localization_hand                  0
localization_lower extremity       0
localization_neck                  0
localization_scalp                 1
localization_trunk                 0
localization_upper extremity       0
sex_female                         0
sex_male                           1
Name: 0, dtype: object

In [12]:
train,test=train_test_split(preprocessed_data)


In [13]:
train.to_csv("../dataset/preprocessed/ham1000_train.csv",index=False)
test.to_csv("../dataset/preprocessed/ham1000_test.csv",index=False)

In [14]:
train.head()

,image_id,dx,age,localization_abdomen,localization_acral,localization_back,localization_chest,localization_ear,localization_face,localization_foot,localization_genital,localization_hand,localization_lower extremity,localization_neck,localization_scalp,localization_trunk,localization_upper extremity,sex_female,sex_male
844,ISIC_0028094,0,70.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4547,ISIC_0029816,0,35.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
6692,ISIC_0028988,0,25.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1
2936,ISIC_0028687,0,80.0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
7420,ISIC_0033014,0,40.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [16]:
Counter(train['dx'])

Counter({0: 6506, 1: 814})

# Combine menlanoma ana bengin dataset for features extractor model

In [7]:
melanoma_folder_path="../dataset/preprocessed/mel_combined_dataset/melanoma"
bengin_folder_path="../dataset/preprocessed/mel_combined_dataset/bengin"

In [16]:


for i in range(len(preprocessed_data)):
    img_name,label=preprocessed_data.iloc[i][0],preprocessed_data.iloc[i][1]
    if label==1:
        copyfile_to_dir(f"{ham_img_dataset_path}/{img_name}.jpg",melanoma_folder_path)
    else:
        copyfile_to_dir(f"{ham_img_dataset_path}/{img_name}.jpg",bengin_folder_path)

In [6]:
combined_path="../dataset/preprocessed/mel_combined_dataset"

In [4]:
train_transform=transforms.Compose([transforms.ToTensor(),transforms.RandomHorizontalFlip(),transforms.RandomRotation(10),Normalize()])


In [8]:
combined_data=MelanomaDataset(combined_path,{"melanoma":1,"bengin":0},train_transform)

In [6]:
train_melanoma,test_melanoma=train_test_split()

# Split features model Dataset
 1. create Dataframe has all melanoma and non-melanoma images
 2. save csv file containts img_name and it's label
 3. copy all images in one folder
 4. split data to train-datafram and test-dataframe
 5. save train_df and test_df to csv file

In [6]:
raw_dataset_path="../dataset/preprocessed/mel_combined_dataset"
clases=os.listdir(raw_dataset_path)
print(clases)


['bengin', 'melanoma']


In [15]:
full_dataset_rows=[]
new_full_imgs_folder=f"{raw_dataset_path}/images"
os.mkdir(new_full_imgs_folder)

In [18]:
for cl in clases:
    img_list=os.listdir(f"{raw_dataset_path}/{cl}")
    class_label=0
    if cl == "melanoma":
        class_label=1
    for img_name in img_list:
        full_dataset_rows.append([img_name,class_label])
        copyfile_to_dir(f"{raw_dataset_path}/{cl}/{img_name}",new_full_imgs_folder)


In [21]:
import random
random.shuffle(full_dataset_rows)
full_data_df=pd.DataFrame(full_dataset_rows,columns=["Image Name","Label"])

In [22]:
full_data_df.head()


,Image Name,Label
0,AUG_0_5069.jpeg,1
1,ISIC_0030671.jpg,0
2,ISIC_0027768.jpg,0
3,AUGmented_0_3964.jpeg,1
4,ISIC_0032306.jpg,0


In [29]:
train_df,test_df=train_test_split(full_data_df)

In [30]:
train.count()

Image Name    12873
Label         12873
dtype: int64

In [32]:
train.groupby('Label').count()

,Image Name
Label,
0,6454
1,6419


In [31]:
test_df.count()

Image Name    4291
Label         4291
dtype: int64

In [33]:
test_df.groupby('Label').count()

,Image Name
Label,
0,2171
1,2120


In [34]:
train_df.to_csv(f"{raw_dataset_path}/train.csv",index=False)
test_df.to_csv(f"{raw_dataset_path}/test.csv",index=False)

In [10]:
full_imgs_folder="../dataset/raw/all_images"
full_imgs_prep_folder="../dataset/preprocessed/all_images"

In [8]:
os.mkdir(full_imgs_prep_folder)

In [ ]:
from torchvision.transforms.transforms import  Resize
from src.dataset import data_preprocessing
data_preprocessing(full_imgs_folder,full_imgs_prep_folder,Resize((240,240)))